In [1]:
%load_ext cython

## Calling C++ libraries from Cython

We can compile a C++ file alongside our actual program.

In [2]:
%%cython
#distutils: language=c++
#distutils: include_dirs=D:/Programming/Python/Cython/CythonBuild

cdef extern from 'Randoo.cpp':
    cdef int randomInt(int)

cpdef int findRandomNumber(int n):
    return randomInt(n) + 1

In [4]:
findRandomNumber(10)

5

In [5]:
!python CythonBuild/setupCompileWithCpp.py build_ext --inplace

Compiling CythonBuild/ExampleCpp.pyx because it changed.
[1/1] Cythonizing CythonBuild/ExampleCpp.pyx
running build_ext
building 'ExampleCpp' extension
D:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -ID:/Programming/Python/Cython/CythonBuild -ICythonBuild -ID:\Anaconda3\include -ID:\Anaconda3\include "-ID:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\ATLMFC\include" "-ID:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.16.27023\include" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17763.0\winrt" "-IC:\Program Files (x86)\Windows Kits\10\includ

D:\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: D:\Programming\Python\Cython\CythonBuild\ExampleCpp.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


## Use a fused type instead of a template

In [36]:
%%cython
#distutils: language=c++

cimport cython

ctypedef fused long_or_double:
    cython.long
    cython.double

cpdef long_or_double cMin(long_or_double a, long_or_double b):
    return min(a, b)

In [37]:
cMin(1, 2)

1

## The STL

C++ has a standard template library which can be imported into Cython.

The stl map type can be used in place of a python dict. The api is not as smooth as Python. It isn't faster either...

In [124]:
%%cython
#distutils: language=c++

from libcpp.vector cimport vector
from libcpp.string cimport string
from libcpp.map cimport map
from libcpp.pair cimport pair

ctypedef pair[string, int] dPair

cpdef func():
    cdef vector[int] v
    v.push_back(10)
    return v

cpdef periodicTable():
    cdef map[string, int] table
    cdef pair[string, int] helium = (b'He', 2)
    table.insert(helium)

    cdef int i    
    for i in range(255):
        table.insert((b'H' + bytes([i]), 1))
    return table

cpdef cyPeriodTable():
    output = {b'He': 2}
    for i in range(255):
        output[b'H' + bytes([i])] = 1
    return output

In [ ]:
periodicTable()

In [116]:
%timeit periodicTable()

195 µs ± 4.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [117]:
def pyPeriodTable():
    output = {b'He': 2}
    for i in range(255):
        output[b'H' + bytes([i])] = 1
    return output

In [118]:
%timeit pyPeriodTable()

114 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Just using a dictionary in cython is faster and not using the int type for the loop. Python dictionaries are relatively fast.

In [125]:
%timeit cyPeriodTable()

88.1 µs ± 4.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## C++ sorting

In [131]:
%%cython
#distutils: language=c++

from libcpp.vector cimport vector
from cython.operator cimport dereference as deref

cdef extern from "<algorithm>" namespace "std":
    void std_sort "std::sort" [iter](iter first, iter last)

def sortList(list list_vec):
    cdef vector[int] vec = list_vec
    std_sort[vector[int].iterator](vec.begin(), vec.end())
    return vec

In [132]:
sortList([2, 1, 4, 5, 0])

[0, 1, 2, 4, 5]

In [134]:
%timeit sortList([2, 1, 4, 5, 0])

1.31 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


This is a few times slower than using a C implementation which has an easier time copying over variables.